# Problem Statement
Given a set of items $I$, a set of votes $V$, a classification function $fn$, a classification threshold $th$ and a cost ratio for crowd to expert vote cost $cr$, for each item we want to find the minimum amount of votes needed to take the decision of continue collecting votes or swith to an expert vote. For this we describe a smart stopping algorithm. <br>
**We define a 3 methods structure**:
- the **classification fn** which returns the probability of an item being classified
- the **cost estimator** which returns the estimated cost for each item given the votes
- the **decision function** which returns a boolean decision for each item

In [1]:
#imports
import numpy as np
import collections
import random
from tqdm import tqdm
import pandas as pd
#import cf's libs
from helpers.mv_single_binary import majority_voting
from helpers.algorithms_utils import input_adapter_single

rationale for the algorithm

In [2]:
# max 20 lines
'''
    Input:
        v - votes for item i
        ct - value between 0 and 1 for deciding if prob of data is enough or must continue
        cf - function to calculate how likely is to be classified
        cr - cost ratio between crowd to expert vote [0,1]
    Output:
        (cost_mean, cost_std)
'''
def cost_estimator(v, ct, cf, cr):
    actual_cost, expert_cost, must_continue = len(v) * cr, 1/cr, True
    simulated_costs = []
    for _ in range(drawing_simulations_amount):            
        while (must_continue == True):
            classification_prob = cf(input_adapter_single(v))
            if classification_prob > ct:
                must_continue = False            
                simulated_costs.append(actual_cost)
            else:
                vote = np.random.binomial(1, classification_prob)
                new_index = max(v.keys()) + 1
                v[new_index] = [vote]
                actual_cost += 1
                if(actual_cost >= (expert_cost * expert_cost_increment)):
                    must_continue = False
                    simulated_costs.append(actual_cost)

    return (np.mean(simulated_costs),np.std(simulated_costs))

In [30]:
# max 20 lines
'''
Function to answer: must continue collecting votes over each task?

Input:
items - set of items
votes - set of votes over each item
classification_threshold - value between 0 and 1 for deciding if prob of data is enough or must continue
cost_ratio - ratio of crowd to expert cost, [0,1]
classification_function - function to calculate how likely is to be classified

Output:
    Dictionary with the decision indexed by item_id
        {
            item_id: bool
            ...
            item_n: ...
        }
    Where False = Stop and True=Continue collecting votes
'''
def decision_function(items, votes, classification_threshold, cost_ratio, classification_function):      
    expert_cost = 1 / cost_ratio  
    results = dict.fromkeys(range(items), False)

    for item_id in range(items):            
        item_votes = votes[item_id].copy()
        actual_cost = len(item_votes) 
        classification_prob = classification_function(input_adapter_single(item_votes))
        if classification_prob <= classification_threshold:
            cost_mean, cost_std = cost_estimator(item_votes, classification_threshold, classification_function, cost_ratio)

            if(cost_mean <= expert_cost):
                results[item_id] = True

    return results

# Experiments

Here we discuss a few experiments, the objective is to compare the overall crowdsourcing cost and quality in the case where we have a smart stopping algorithm vs 
- the baseline approach where all items receive the same amount of votes
- an xx approach where you ask like 2 votes, if they disagree ask a third to break the tie

## Basic settings: MV as classification function, Expected cost limited by expert cost

**Assumptions**:
- The items are evaluated over only 1 condition
- Difficulty of tasks are all equal
- There are no test questions
- The crowd workers accuracy is fixed

In [25]:
#hyperparameters
#main 
cf = majority_voting
ct = .9
cr = .01 #ratio 1:100
base_votes_per_item = 3

#cost estimator 
drawing_simulations_amount = 50
expert_cost_increment = 2

#crowd
workers_num = 1000
z = 0 #% cheaters
fixed_acc = True
workers_acc = .9

#ground truth 
items_num = 100
data_true_percentage = 1

#experiment 
cts = np.arange(.7, .96, 0.05) #classification thresholds
iterations_per_ct = 50

In [14]:
# simulate workers
def simulate_workers(workers_num, cheaters_prop, fixed_acc, workers_acc):
    workers = []
    for _ in range(workers_num):
        if (fixed_acc == False):
            if np.random.binomial(1, cheaters_prop):
                # worker_type is 'rand_ch'
                worker_acc_pos = worker_acc_neg = 0.5
            else:
                # worker_type is 'worker'
                worker_acc_pos = 0.5 + (np.random.beta(1, 1) * 0.5)
                worker_acc_neg = worker_acc_pos + 0.1 if worker_acc_pos + 0.1 <= 1. else 1.
        else:
            worker_acc_pos = workers_acc
            worker_acc_neg = worker_acc_pos + 0.1 if worker_acc_pos + 0.1 <= 1. else 1.

        workers.append([worker_acc_pos, worker_acc_neg])

    return workers

In [45]:
'''
    items_num - number of items
    possitive_percentage - [0,1] percentage of possitive items
'''
def generate_gold_data(items_num, possitive_percentage):
    pos_items_number = int(round(((possitive_percentage * 100) * items_num) / 100))     
    gold_data = ([1] * pos_items_number) + ([0] * (items_num - pos_items_number))
    random.shuffle(gold_data)

    return gold_data

def classify_items(votes, gt, cf, th):
    items_classification = {}
    for i, v in votes.items():
        prob = cf(input_adapter_single(v))
        if (prob > th):
            if(gt[i] == 1):
                items_classification[i] = 1
            else:
                items_classification[i] = 0
        else:
            items_classification[i] = gt[i] #if didnt reach the threshold, switch to expert vote

    return items_classification

In [59]:
class Metrics:

    @staticmethod
    def compute_metrics(items_classification, gt):
        gt_scope = []

        # FP == False Inclusion
        # FN == False Exclusion
        fp = fn = tp = tn = 0.
        for item_id in range(len(gt)):
            gt_val = gt[item_id]
            cl_val = items_classification[item_id]
            if gt_val and not cl_val:
                fp += 1
            if not gt_val and cl_val:
                fn += 1
            if gt_val and cl_val:
                tn += 1
            if not gt_val and not cl_val:
                tp += 1
        if (tp + fn > 0):
            recall = tp / (tp + fn)
            precision = tp / (tp + fp)
        else:
            recall = tp
            precision = tp
        loss = (fp + fn) / len(gt)
        
        return loss,  recall, precision

In [43]:
class Generator:

    def __init__(self, params):
        self.workers_accuracy = params['workers_accuracy']
        self.workers_num = params['workers_num']      
        self.items_num = params['items_num']      
        self.cost_ratio = params.get('cost_ratio')
        self.classification_threshold = params.get('classification_threshold')
        self.index_workers_voted_on_item = {}
        self.votes_per_item = params['votes_per_item']
        self.classification_fn = params['classification_fn']
    
    def get_random_worker_accuracy(self, item, items_num):
        #TO-DO
        '''
        worker_found = False  
        while (worker_found == False):
            worker_id = random.randint(0, self.workers_num - 1)
            if (item in self.index_workers_voted_on_item.keys()):
                if (worker_id not in self.index_workers_voted_on_item[item]):
                    self.index_workers_voted_on_item[item].append(worker_id)
                    worker_found = True
            else:
                worker_found = True
                self.index_workers_voted_on_item[item] = [worker_id]
        '''
       
        worker_id = random.randint(0, self.workers_num - 1)
        worker_acc_pos = self.workers_accuracy[worker_id][0]
        worker_acc_neg = self.workers_accuracy[worker_id][1]
        return {'worker_id': worker_id, 'acc_pos':worker_acc_pos, 'acc_neg': worker_acc_neg}
    
    def get_worker_vote(self, i, gt, items_num):
        worker_data = self.get_random_worker_accuracy(i, items_num)
        worker_id, worker_acc_pos, worker_acc_neg = worker_data['worker_id'], worker_data['acc_pos'], worker_data['acc_neg']
        
        item_is_pos = gt[i] == 1
        
        if (item_is_pos):
            worker_acc = worker_acc_pos
        else:
            worker_acc = worker_acc_neg
        
        if np.random.binomial(1, worker_acc):
            vote = gt[i]
        else:
            vote = 1 - gt[i]
            
        return (worker_id, vote)
    
    def get_items_predicted_classified(results):
        return {i:v for (i,v) in results.items() if v == True}
    
    def generate_votes(self, items_num, ct, gt):
        total_votes = {}
     
        #base votes
        for i in range(items_num):
            total_votes[i] = {}
            for k in range(self.votes_per_item):
                worker_id, vote = self.get_worker_vote(i, gt, items_num)

                total_votes[i][worker_id] = [vote]
                
        #evaluate votes
   
        results = decision_function(items_num, total_votes, ct, self.cost_ratio, 
                                                       self.classification_fn)
        #Check if must continue collecting votes
        items_predicted_classified = Generator.get_items_predicted_classified(results)
        must_get_more_votes = len(items_predicted_classified) > 0

        while(must_get_more_votes):
            for i, v in items_predicted_classified.items():
                worker_id, vote = self.get_worker_vote(i, gt, items_num)

                total_votes[i][worker_id] = [vote]             
            #end for
            results = decision_function(items_num, total_votes, ct, self.cost_ratio, 
                                                       self.classification_fn)
            
            #Stop when there are no more items that can be classified
            items_predicted_classified = Generator.get_items_predicted_classified(results)
            must_get_more_votes = len(items_predicted_classified) > 0
        #end while
        
        items_classification = classify_items(total_votes, gt, self.classification_fn, ct)
            
        return [items_classification, total_votes]

In [60]:
main_results = []

workers_accuracy = simulate_workers(workers_num, z, fixed_acc, workers_acc)

params = {
    'workers_accuracy': workers_accuracy,
    'workers_num': workers_num,
    'items_num': items_num,
    'cost_ratio': cr,
    'votes_per_item': base_votes_per_item,
    'classification_fn': cf
}

for ct in tqdm(cts):
    ct = round(ct, 2) #limit to two decimals
    crowd_cost = []
    items_classified_in = []
    items_classified_out = []
    ct_loss = []
    ct_recall = []
    ct_precision = []
    
    for _ in range(iterations_per_ct):
        ground_truth = generate_gold_data(items_num, data_true_percentage)
       
        ct_i_results = Generator(params).generate_votes(items_num, ct, ground_truth)
        
        items_classification = ct_i_results[1]
        total_votes = ct_i_results[1]

        crowd_cost.append(sum([len(v) for (x,v) in th_total_votes.items()]) * cr)
        
        loss,  recall, precision = Metrics.compute_metrics(items_classification, ground_truth)
        ct_loss.append(loss)
        ct_recall.append(recall)
        ct_precision.append(precision)
    #end for iterations
    
    main_results.append(
        [ct, round(np.mean(crowd_cost), 3), 
         round(np.std(crowd_cost), 3), 
         round(np.mean(crowd_cost) / items_num, 3),
         np.mean(ct_loss),
         np.std(ct_loss),
         np.mean(ct_recall),
         np.std(ct_recall),
         np.mean(ct_precision),
         np.std(ct_precision)
        ])
#end for thresholds

pd.DataFrame(main_results, columns=["Threshold","Cost Avg","Cost Std", "Cost per item", "Loss", "Loss Std", "Recall", "Recall Std", "Precision", "Precision Std"])

100%|██████████| 6/6 [13:43<00:00, 196.45s/it]


,Threshold,Cost Avg,Cost Std,Cost per item,Loss,Loss Std,Recall,Recall Std,Precision,Precision Std
0,0.70,3.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0
1,0.75,3.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0
2,0.80,3.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0
3,0.85,3.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0
4,0.90,3.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0
5,0.95,3.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0
